In [ ]:
# api 설치
%pip install -r ../requirements.txt

In [ ]:

from fastapi import FastAPI
from pydantic import BaseModel
from contextlib import asynccontextmanager
from module.model import load_models, predict

ModuleNotFoundError: No module named 'classification.model'

In [ ]:
# lifespan으로 모델 로드 관리
@asynccontextmanager
async def lifespan(app: FastAPI):
    global models, tokenizers, device
    models, tokenizers, device = load_models()
    yield
    models.clear()
    tokenizers.clear()

In [ ]:
app = FastAPI(title="KLUEBERT Multi-Model API", lifespan=lifespan)

In [ ]:
class PredictInput(BaseModel):
    text: str

# 분석조회 테이블 출력
# 현재 감지모델 출력만 있음

@app.post("/predict")
def get_prediction(data: PredictInput):
    results = {}
    for disease in ["depression", "anxiety"]:
        results[disease] = predict(data.text, models[disease], tokenizers[disease], device)
    return {"input": data.text, "predictions": results}

In [ ]:
# 직동확인 부분
@app.get("/")
def root():
    return {"message": "KLUEBERT API is running", "models": ["depression", "anxiety"]}